# Azure Assistant Tracing

This notebook demonstrates how we can instrument an Azure Assistant for tracing and feedback within an application. While done in a notebook there is no need to run this within Databricks, since it is designed to slot into a streamlit application.

In [ ]:
import time
from openai import AzureOpenAI
import mlflow
from databricks.sdk import WorkspaceClient
from mlflow.tracing.destination import Databricks

The first thing we do is setup some basic authentication. I am using a [default WorkspaceClient authentication](https://docs.databricks.com/aws/en/dev-tools/sdk-python#authenticate-the-databricks-sdk-for-python-with-your-databricks-account-or-workspace), but you may need to pass a host and token in your app or use [OAuth](https://docs.databricks.com/aws/en/dev-tools/auth/oauth-u2m) (recommended). If using OAuth, we can use the workspace client to get the user's information for conversation history and tracing.

In [185]:
w = WorkspaceClient()
endpoint = "https://dbmma.openai.azure.com/"
api_key = w.dbutils.secrets.get(scope="shm", key="azure_agent_key")

# use this to get Databricks credentials for tracking
user_name = w.current_user.me().user_name
user_id = w.current_user.me().id

The first concept is to use an experiment to track our application. I would recommend either using one experiment for the whole application (i.e. Agent) and tagging each release, or using one experiment per release (i.e. Agent-V1, Agent-V2, etc).

In [186]:
# experiment = w.experiments.create_experiment('/Users/scott.mckean@databricks.com/experiments/azure_assistant').experiment

experiment = w.experiments.get_by_name(
  '/Users/scott.mckean@databricks.com/experiments/azure_assistant'
  ).experiment

This follows the standard Azure authentication and inference flow. One critical note here is that the Azure Assistant API only works with the AzureOpenAI Assistants API, which is now deprecated and being replaced by the ResponsesAPI. It should be fine for the POC though.

In [187]:
client = AzureOpenAI(
  azure_endpoint = endpoint,
  api_key= api_key,
  api_version="2024-05-01-preview"
)

We can either create a new assistant using the assistants.create() method or use an existing one using the assistants.retrieve() method.

In [188]:
assistant = client.beta.assistants.retrieve(assistant_id='asst_Mbf4tcs8E6ZOv4aOQ9nKBy7o')

# or 
# assistant = client.beta.assistants.create(
#   model="gpt-4o-mini", # replace with model deployment name.
#   instructions="You are an assistant that answers questions about the FORGE geothermal project in Utah.",
#   tools=[{"type":"file_search"}],
#   tool_resources={"file_search":{"vector_store_ids":["vs_VhOIWdzPecXrOlDMWmlhhUvH"]}},
#   temperature=0.17,
#   top_p=0.1
# )

We set the tracking server to Databricks and reference the experiment ID. This should be done in the application (could use environment variables for the experiment ID).

In [16]:
mlflow.set_tracking_uri("databricks")
mlflow.tracing.set_destination(
    Databricks(experiment_id=experiment.experiment_id)
    )

This is the heart of the tracing code. We wrap the default assistant code with a mlflow.start_span() context manager. Now that we have a span, we can set inputs, attributes, trace metadata, and log the outputs. 

It is worth noting the hierarchy here:
Experiment -> Trace -> Span -> Assistant Calls

I've done some basic quality of life conversions here to make sure we have the messages easily accessible, and the proper metadata captured, the two most important ones being the session_id and user_id, which we will need to match with our conversation history capture in Postgres or Cosmos.

In [189]:
def run_assistant_query(client, assistant, user_name, question: str):

    with mlflow.start_span(
        name='azure_assistant_query', 
        span_type='LLM'
    ) as span:
        thread = client.beta.threads.create()      
        
        span.set_inputs({
            'messages': [{'role': 'user', 'content': question}],
            "user_id": user_id,
            "assistant_id": assistant.id,
            "thread_id": thread.id,
        })
  
        mlflow.update_current_trace(
            client_request_id=thread.id,
              metadata={
                  "mlflow.trace.user": user_name,
                  "mlflow.trace.session": thread.id, 
              }
          )
  
        # Add a user question to the thread
        _ = client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=question,
        )

        # Run the thread
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id,
        )

        # Poll until completion
        while run.status in ["queued", "in_progress", "cancelling"]:
            time.sleep(1)
            run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
            )

        # Capture status and AzureOpenAI attributes
        mlflow.update_current_trace(state="OK")
        span.set_attributes(run.model_dump())    
        
        # Extract a standard dictionary of assistant responses
        # Currently doens't include tools (build on Responses API later)
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
            )

        output_messages = [
            {
                'role': x.role,
                'content': x.content[0].text.value
            } 
            for x in messages.data
        ]

        all_outputs = [
            x.model_dump() 
            for x in messages.data 
            if x.role == 'assistant'
            ]
        
        # Log agent response as MLflow output
        span.set_outputs({
            'messages': output_messages,
            'output': all_outputs
            })

        
        return {
            'messages':output_messages, 
            'output':all_outputs,
            'attributes':run.model_dump()
            }

Now we run the assistant - the tracing is automatically captured, but we get the result back so we can pull the trace and return the assistant message.

In [191]:
# Single traced call; print concise outputs
question = "What wells are on the project?"
result = run_assistant_query(client, assistant, user_name, question)

/var/folders/2x/pf13chqx4614qjlmjdgmndv00000gp/T/ipykernel_48710/2802846583.py:7: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
/var/folders/2x/pf13chqx4614qjlmjdgmndv00000gp/T/ipykernel_48710/2802846583.py:25: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  _ = client.beta.threads.messages.create(
/var/folders/2x/pf13chqx4614qjlmjdgmndv00000gp/T/ipykernel_48710/2802846583.py:32: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
/var/folders/2x/pf13chqx4614qjlmjdgmndv00000gp/T/ipykernel_48710/2802846583.py:40: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(
/var/folders/2x/pf13chqx4614qjlmjdgmndv00000gp/T/ipykernel_48710/2802846583.py:51: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  me

With the experiment ID and trace ID, we can directly query the trace and get the UUID from mlflow. We can also pull all of the user's previous threads for helping with conversation history or feedback.

In [196]:
# search a user's previous threads
previous_threads =mlflow.search_traces(
    experiment_ids=[experiment.experiment_id],
    order_by=['timestamp DESC'],
    filter_string=f"metadata.mlflow.trace.user = '{user_name}'",
    max_results=3
)

previous_threads

,trace_id,trace,client_request_id,state,request_time,execution_duration,request,response,trace_metadata,tags,spans,assessments
0,tr-e9fffec78fe06a9b840e90137fe6132e,"{""info"": {""trace_id"": ""tr-e9fffec78fe06a9b840e...",thread_M8xCSmNpeCHmFPl0eUEgKfi5,TraceState.OK,1756269415149,9099,"{'messages': [{'role': 'user', 'content': 'Wha...","{'messages': [{'role': 'assistant', 'content':...",{'mlflow.source.git.commit': 'f46d2c154c81f3ea...,{'mlflow.artifactLocation': 'dbfs:/databricks/...,"[{'trace_id': '6f/+x4/gapuEDpATf+YTLg==', 'spa...",[]
1,tr-2d992b0a474c63563de51d064ec83817,"{""info"": {""trace_id"": ""tr-2d992b0a474c63563de5...",thread_mESfDO8fIDWEATF9G2dmaDUk,TraceState.OK,1756268267832,8061,"{'messages': [{'role': 'user', 'content': 'Wha...","{'messages': [{'role': 'assistant', 'content':...",{'mlflow.source.git.commit': 'f46d2c154c81f3ea...,{'mlflow.artifactLocation': 'dbfs:/databricks/...,"[{'trace_id': 'LZkrCkdMY1Y95R0GTsg4Fw==', 'spa...",[{'assessment_id': 'a-a1083f7f0bab4ccb83546ad4...
2,tr-a4c4bbbd91bbcf7f913985662c1c6385,"{""info"": {""trace_id"": ""tr-a4c4bbbd91bbcf7f9139...",thread_FWbiBExYTbueu5CtBvDSGxMs,TraceState.OK,1756268212396,11390,"{'messages': [{'role': 'user', 'content': 'Wha...","{'messages': [{'role': 'assistant', 'content':...",{'mlflow.source.git.commit': 'f46d2c154c81f3ea...,{'mlflow.artifactLocation': 'dbfs:/databricks/...,"[{'trace_id': 'pMS7vZG7z3+ROYVmLBxjhQ==', 'spa...",[]


In [200]:
import urllib.request
import tarfile

# Download the file
url = "https://amazon-visual-anomaly.s3.amazonaws.com/VisA_20220922.tar"
file_path = "/tmp/VisA_20220922.tar"
urllib.request.urlretrieve(url, file_path)

# Extract the tar file directly into the volume
with tarfile.open(file_path) as tar:
    tar.extractall(path="./pcb_training_data")

In [ ]:
# retrieve the mlflow trace id for feedback
thread_id = result['attributes']['thread_id']

# get the specific trace
mlflow_trace_id =mlflow.search_traces(
    experiment_ids=[experiment.experiment_id],
    order_by=['timestamp DESC'],
    filter_string=f"tag.user_name = '{user_name}'",
    max_results=3
).iloc[0].trace_id

Once we have the trace ID, adding Human feedback with our application is straightforward - see an example of a boolean (thumbs up/down), a numeric score, and [LLM evaluation](https://mlflow.org/docs/2.21.3/llms/llm-evaluate/notebooks/question-answering-evaluation#custom-llm-judged-metric-for-professionalism).

We can also set [expectations](https://mlflow.org/docs/latest/genai/assessments/expectations/#types-of-expectations). This can be very useful for the future if we want to use something like [DsPy and MLfLow to optimize our prompts](https://mlflow.org/docs/latest/genai/prompt-registry/optimize-prompts/).

In [ ]:
import mlflow
from mlflow.entities import AssessmentSource, AssessmentSourceType

# thumbs up/down button
mlflow.log_feedback(
    trace_id=mlflow_trace_id,
    name="user_satisfaction",
        value=True,
        source=AssessmentSource(
            source_type=AssessmentSourceType.HUMAN, source_id=user_name
        ),
    )

# numeric score
mlflow.log_feedback(
    trace_id=mlflow_trace_id,
    name="relevance",
    value=0.9,
    source=AssessmentSource(
        source_type=AssessmentSourceType.HUMAN, source_id=user_name
    ),
    rationale="High accuracy and clarity, slightly incomplete coverage",
)

# llm evaluation (could be done with a job)
mlflow.log_feedback(
    trace_id=mlflow_trace_id,
    name="llm_evaluation",
    value=0.3,
    source=AssessmentSource(
        source_type=AssessmentSourceType.LLM_JUDGE, source_id='GPT-5'
    ),
    rationale="Not as good as I could have done",
)

Feedback(name='llm_evaluation', source=AssessmentSource(source_type='LLM_JUDGE', source_id='GPT-5'), trace_id='tr-e9fffec78fe06a9b840e90137fe6132e', run_id=None, rationale='Not as good as I could have done', metadata=None, span_id=None, create_time_ms=1756269565921, last_update_time_ms=1756269565921, assessment_id='a-0e4280febbda41eea6699030bd334d6b', error=None, expectation=None, feedback=FeedbackValue(value=0.3, error=None), overrides=None, valid=True)

In [199]:
mlflow.log_expectation(
    trace_id=mlflow_trace_id,
    name="expected_behavior",
    value={
        "should_escalate": True,
        "required_elements": ["empathy", "solution_offer", "follow_up"],
        "max_response_length": 150,
        "tone": "professional_friendly",
    },
    source=AssessmentSource(
        source_type=AssessmentSourceType.HUMAN,
        source_id=user_name,
    ),
)

Expectation(name='expected_behavior', source=AssessmentSource(source_type='HUMAN', source_id='scott.mckean@databricks.com'), trace_id='tr-e9fffec78fe06a9b840e90137fe6132e', run_id=None, rationale=None, metadata=None, span_id=None, create_time_ms=1756270787382, last_update_time_ms=1756270787382, assessment_id='a-80ac5b0169bb493bb18d60b1e96b5485', error=None, expectation=ExpectationValue(value={'should_escalate': True, 'required_elements': ['empathy', 'solution_offer', 'follow_up'], 'max_response_length': 150, 'tone': 'professional_friendly'}), feedback=None, overrides=None, valid=None)